In [2]:
import thulac
import jieba
import re
import pandas as pd

In [3]:
Loc=thulac.thulac()

Model loaded succeed


In [5]:
#编写的thulac的切分词函数，得到词，标签，起始位置和结束位置
def tokenize_thu(unicode_sentence):
    
    """
    Tokenize a sentence and yields tuples of (word, tag, start, end)
    Parameter:
        - sentence: the str(unicode) to be segmented.
    """
    start = 0
    for i in Loc.cut(unicode_sentence):
        w = i[0]
        tag = i[1]
        width = len(w)
        yield (w, tag, start, start + width)
        start += width
        
#example

a=tokenize_thu('2010年4月20日首次在江苏省如东太阳岛以东海域发现零星漂浮浒苔。')
#a=tokenize_thu('深夜的穆赫兰道发生一桩车祸，女子丽塔在车祸中失忆了')
for i,j,k,l in a:
    print(i,j,k,l)

深夜 t 0 2
的 u 2 3
穆赫兰道 nz 3 7
发生 v 7 9
一 m 9 10
桩 q 10 11
车祸 n 11 13
， w 13 14
女子 n 14 16
丽塔 n 16 18
在 p 18 19
车祸 n 19 21
中 f 21 22
失忆 v 22 24
了 u 24 25


In [1]:
import gensim

In [2]:
texts = [['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [3]:
from gensim import corpora

In [4]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [5]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

from pprint import pprint  # pretty-printer
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [6]:
dictionary = corpora.Dictionary(texts)
#dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))  # store the dictionary, for future reference
print(dictionary)

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [7]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [8]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored

[(0, 1), (1, 1)]


In [22]:
corpus = [dictionary.doc2bow(text) for text in texts]
#corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus)  # store to disk, for later use
for c in corpus:
    print(c)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [23]:
from gensim import models
tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model

In [24]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])  # step 2 -- use the model to transform vectors

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [25]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
     print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [39]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation

In [40]:
corpus_lsi = lsi[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

In [41]:
lsi.print_topics(2)

[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [74]:
documents = ['工业互联网平台的核心技术是什么',
            '工业现场生产过程优化场景有哪些']

In [63]:
def word_cut(doc):
    seg = [jieba.lcut(w) for w in doc]
    return seg

In [ ]:
def StopWordsList(filepath):
    wlst = [w.strip() for w in open(filepath,'r',encoding='utf8').readlines()]
    return wlst

In [ ]:
stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']

In [81]:
#对文本集中的文本进行中文分词，返回分词列表
def seg_sentence(sentence,stop_words):
    sentence_seged = jieba.cut(sentence.strip())
    # sentence_seged = set(sentence_seged)
    outstr = ''
    for word in sentence_seged:
        if word not in stop_words:
            if word != '\t':
                outstr += word
                outstr += ' '

    return outstr.rstrip().split(' ')


In [96]:
stop_words=['的','是','也','有']
sen='工业互联网平台的核心技术是什么'
seg_sentence(sen,stop_words)

['工业', '互联网', '平台', '核心技术', '什么']

In [97]:
texts=[seg_sentence(seg,stop_words) for seg in documents]

In [98]:
#2、基于文件集建立【词典】，并提取词典特征数
dictionary = corpora.Dictionary(texts)
feature_cnt = len(dictionary.token2id.keys())

In [100]:
#3、基于词典，将【分词列表集】转换为【稀疏向量集】，也就是【语料库】
corpus = [dictionary.doc2bow(text) for text in texts]

In [101]:
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(2, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]]

In [103]:
#4、使用“TF-TDF模型”处理【语料库】
tfidf = models.TfidfModel(corpus)

In [64]:
texts=word_cut(documents)

In [102]:
##为语料库中出现的所有单词分配了一个唯一的整数id
dictionary.token2id

{'互联网': 0,
 '什么': 1,
 '工业': 2,
 '平台': 3,
 '核心技术': 4,
 '优化': 5,
 '哪些': 6,
 '场景': 7,
 '现场': 8,
 '生产': 9,
 '过程': 10}

In [67]:
##该函数doc2bow()只计算每个不同单词的出现次数，将单词转换为整数单词id，并将结果作为稀疏向量返回
bow_corpus = [dictionary.doc2bow(text) for text in texts]

In [68]:
bow_corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(2, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]]

In [70]:
from gensim import models
# train the model
tfidf = models.TfidfModel(bow_corpus)

TypeError: 'NoneType' object is not callable